<a href="https://colab.research.google.com/github/SumaiaAnjumShaba/CSE710-Advanced-Artificial-Intelligence/blob/main/24366030_CSE710_P2_HMM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Write a program to determine learning parameters of HMM using unsupervised learning (Baum Welch algorithm) considering the following dataset. You have to write codes for the necessary algorithms, the use of library functions are not allowed.

In [ ]:
import numpy as np

observations_sequences = [
    ['R', 'R', 'D', 'D', 'R', 'D', 'R'],
    ['D', 'R', 'D', 'D', 'R', 'R', 'D'],
    ['R', 'R', 'D', 'D', 'R', 'D', 'R'],
    ['R', 'D', 'D', 'D', 'D', 'R', 'R'],
    ['D', 'R', 'D', 'D', 'D', 'R', 'R']
]

obs_map = {'R': 0, 'D': 1} #Mapping if R then map  to 0 and D then map  to 1

number_states = 2  # H, L

num_obs = len(observations_sequences[0]) # Number of observations

np.random.seed(42)  # it will generate same number every time when run the code


pi = np.random.rand(number_states)

pi /= pi.sum()  # normalizes the values so that they sum to 1

# Transition probability matrix = A (State i to state j)

A = np.random.rand(number_states, number_states) # create 2X2 matrix as states are two H, L

A /= A.sum(axis=1, keepdims=True) # normalized each row

# Emission probability matrix B= (observing k from state i)

B = np.random.rand(number_states, len(obs_map))

B /= B.sum(axis=1, keepdims=True) # normalized each row

print("Generating Initial probability using random number", pi)

print("Initial transition probability matrix:\n", A)

print("Initial emission probability matrix:\n", B)

Generating Initial probability using random number [0.28261752 0.71738248]
Initial transition probability matrix:
 [[0.55010153 0.44989847]
 [0.50003865 0.49996135]]
Initial emission probability matrix:
 [[0.06284339 0.93715661]
 [0.45915117 0.54084883]]


#Forward Algorithm#

In [ ]:
def forward_algorithm(obs_seq, pi, A, B):

    T = len(obs_seq)  # Length of the observation sequence

    N = len(pi)  # Number of states

    alpha = np.zeros((T, N))  # Initialize the alpha table with zeros

    #alpha_1_k(k=l,h state)=initial probabilty(s1=k) * emission probabilty (o1|s1=k)

    first_observation_index_0 = obs_map[obs_seq[0]]  #mapping observation R or D into 0 or 1

    alpha[0, :] = pi * B[:, first_observation_index_0]  #starting with t=0

    for t in range(1, T): #starting t=1 to T

        current_observation_index = obs_map[obs_seq[t]]

        for j in range(N):

            sum_previous_alpha = np.sum(alpha[t-1, :] * A[:, j])

            alpha[t, j] = sum_previous_alpha * B[j, current_observation_index]

            #alpha_k_t=emission probabilty(o_t|st=k) summation of i [(alpha_i_t-1)*transtion probabilty(st=k|st-1=i)]

    return alpha


#Backward Algorithm#

In [ ]:
def backward_algorithm(obs_seq, A, B):

    T = len(obs_seq)

    N = A.shape[0]

    beta = np.zeros((T, N))

    beta[T-1, :] = 1

    for t in range(T-2, -1, -1):

        next_observation_index = obs_map[obs_seq[t+1]]

        for i in range(N):

            beta[t, i] = np.sum(A[i, :] * B[:, next_observation_index] * beta[t+1, :]) #beta= summation of (transtion probability p(st+1=i|st=k) * emission probabilty p(ot+1|st+1=i) * betait+1 )

    return beta

#Beum Welch Algorithm#

In [ ]:
def baum_welch(observation_sequences, pi, A, B, num_iter=5):

    N = len(pi)  # Number of states

    M = len(B[0])  # Number of observation symbols (R and D)

    T = len(observation_sequences[0])  # Length of the sequence

    for n in range(num_iter):

        pi_numerator = np.zeros(N) # N=2 two state H,L

        A_numerator = np.zeros((N, N)) # 2x2 matrix

        A_denominator = np.zeros(N)

        B_numerator = np.zeros((N, M))

        B_denominator = np.zeros(N)


        for obs_seq in observation_sequences:

            alpha = forward_algorithm(obs_seq, pi, A, B)

            beta = backward_algorithm(obs_seq, A, B)


            gamma = np.zeros((T, N))

            eta = np.zeros((T-1, N, N))

            for t in range(T):

                # Normalization factor  (C)
                normalization_factor = np.sum(alpha[t, :] * beta[t, :]) #normaized c= summation of alha and beta

                for i in range(N):

                    gamma[t, i] = alpha[t, i] * beta[t, i] / normalization_factor #gamma= c*alpha*beta

            for t in range(T-1):

                denom = np.sum(alpha[t, :] * np.dot(A, B[:, obs_map[obs_seq[t+1]]]) * beta[t+1, :])

                for i in range(N):

                    for j in range(N):

                        #etaij_t=[(alpha_t_i) * A_i_j * B_j_(z_t+1) * beta_(t+1)_(j)] / [summation of{(alpha_t_k)* A_k_l * b_l_(z_t+1) * beta_t+1(l)}]

                        eta[t, i, j] = alpha[t, i] * A[i, j] * B[j, obs_map[obs_seq[t+1]]] * beta[t+1, j]

                        eta[t, i, j] /= denom

            # Update the HMM parameters

            #update initaial probabilty

            pi_numerator += gamma[0, :]

            for t in range(T-1):

                #update transtion probabilty

                A_numerator += eta[t, :, :]

                A_denominator += np.sum(gamma[t, :], axis=0)

            for t in range(T):

                #update emission probabilty

                B_numerator[:, obs_map[obs_seq[t]]] += gamma[t, :]

                B_denominator += gamma[t, :]



        pi = pi_numerator / len(observation_sequences)

        A = A_numerator / A_denominator[:, None]

        B = B_numerator / B_denominator[:, None]

        print(f"Iteration {n+1}")

        print("Updated Initial state distribution:", pi)

        print("Updated Transition matrix:\n", A)

        print("Updated Emission matrix:\n", B)

        print("\n")

    return pi, A, B



#Main Function Call#

In [ ]:
pi, A, B = baum_welch(observations_sequences, pi, A, B)

print("After appied baum welch algorithm final HMM Parametrs are \n ")
print("Initial Probabilities:", pi)
print("Transition Probabilities:\n",A)
print("Emission Probabilities:\n",B)

Iteration 1
Updated Initial state distribution: [0.1842718 0.8157282]
Updated Transition matrix:
 [[0.16586975 0.23325646]
 [0.23941207 0.36396624]]
Updated Emission matrix:
 [[0.14975051 0.85024949]
 [0.68609032 0.31390968]]


Iteration 2
Updated Initial state distribution: [0.13196833 0.86803167]
Updated Transition matrix:
 [[0.10642339 0.22541537]
 [0.20672173 0.43543421]]
Updated Emission matrix:
 [[0.13743814 0.86256186]
 [0.64316653 0.35683347]]


Iteration 3
Updated Initial state distribution: [0.07383625 0.92616375]
Updated Transition matrix:
 [[0.0439909  0.16949733]
 [0.15960274 0.57178741]]
Updated Emission matrix:
 [[0.12450637 0.87549363]
 [0.58174204 0.41825796]]


Iteration 4
Updated Initial state distribution: [0.02183932 0.97816068]
Updated Transition matrix:
 [[0.0060615  0.07030402]
 [0.07717373 0.74902796]]
Updated Emission matrix:
 [[0.13400726 0.86599274]
 [0.51946065 0.48053935]]


Iteration 5
Updated Initial state distribution: [0.00182814 0.99817186]
Updated Tr